In [1]:
! mamba install hnswlib -y


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.22.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['hnswlib']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64 ━━━━━━╸━━━━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??

In [2]:
!pip install sentence_transformers

  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached sentencepiece-0.1.97-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)


In [3]:
import hnswlib
import numpy as np
from pathlib import Path
import pickle
from IPython.display import Markdown
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
import random
import datetime
import pandas as pd
import re


In [4]:
model_id = "multi-qa-mpnet-base-dot-v1"
model = SentenceTransformer(model_id)

In [5]:
paths = list(Path("segments").glob("**/*.pickle"))

segments = []
for path in tqdm(paths):
    with open(path, "rb") as f:
        segment_subset = pickle.load(f)                    
        segments.extend(segment_subset)

        
duplicated_texts = pd.DataFrame(segments)["text"].value_counts().reset_index()
duplicated_texts_list = duplicated_texts[duplicated_texts["text"] > 2]["index"].tolist()
# filter out segments that appear often in the podcasts. They are not relevant

segments = [x for x in segments 
            if x["text"] not in duplicated_texts_list 
            and "Unfortunately that doesn't exist in this world. We're really at the beginning of the regenerative agriculture" not in  x["text"]
            and "It's also not investment advice." not in x["text"]
            and "So get ready, get a cup of coffee, a cup of tea," not in x["text"]
           ]

dim = 768
N = len(segments)
p = hnswlib.Index(space='l2', dim=dim)  # possible options are l2, cosine or ip
p.init_index(max_elements=N+1, ef_construction=100, M=16)
p.add_items([x.pop("embedding") for x in segments])

  0%|          | 0/390 [00:00<?, ?it/s]

In [6]:
from fastbm25 import fastbm25

tokenized_corpus = [doc["text"].lower().split(" ") for doc in segments]
bm25_model = fastbm25(tokenized_corpus)


ModuleNotFoundError: No module named 'fastbm25'

In [ ]:
def highlight_matches(text, query):
    for word in query.split():
        word = word.replace("?", "").lower()
        text = re.sub(f" {word} ", f" **{word}** ", text, re.IGNORECASE)
        text = re.sub(f" {word.capitalize()} ", f" **{word}** ", text, re.IGNORECASE)
        # text = re.sub(f" {word} ", f" **{word}** ", text, re.IGNORECASE)
    return text
        

def query(query: str, k=10):
    embedding = model.encode(query)
    labels, distances = p.knn_query(embedding, k=k)
    if query.endswith("?"):
        results = [x for idx, x in enumerate(segments) if idx in labels[0]]
    else:
        results = bm25_model.top_k_sentence(query.lower().split(),k=k)
        results = [x for idx, x in enumerate(segments) if idx in [x[1] for x in results]]
    for result, distance in zip(results, distances[0]):
        display(Markdown(f"""{result["meta"]["feed_meta"]["title"]}  
        **{result['meta']['title']}**  
        **{datetime.timedelta(seconds=int(result['start']))}-{datetime.timedelta(seconds=int(result['end']))}**   
        {highlight_matches(result['text'], query)}  
        ---"""
        ))

In [205]:
query("What to do with potatoes?", k=20, ) 

Investing in Regenerative Agriculture and Food  
        **33 Thomas Rippel, using a blockchain to help Farmland Stewardship Organisations grow**  
        **0:34:08-0:34:49**   
        That's very interesting. Yes. I think that's very encouraging and necessary. Well necessary to some extent. In the end you know it is a fact that land prices have gone up substantially in the last especially in the last 20 years. In **germany** well I can give you specific numbers for the last 10 years. So in the past 10 years land prices in **germany** have gone up 150 percent. In neighboring Poland prices have gone up 700 percent. In Romania they've gone up 800 percent.  
        ---

Investing in Regenerative Agriculture and Food  
        **33 Thomas Rippel, using a blockchain to help Farmland Stewardship Organisations grow**  
        **0:34:25-0:35:12**   
        especially in the last 20 years. In **germany** well I can give you specific numbers for the last 10 years. So in the past 10 years land prices in **germany** have gone up 150 percent. In neighboring Poland prices have gone up 700 percent. In Romania they've gone up 800 percent. So it's really really dramatic quite dramatic. And there's you know nothing a farmland stewardship organization can do about these market dynamics. The only thing we can do is to make sure that we buy the land and then it's secured for the long term. Yeah because that's the crux right.  
        ---

Investing in Regenerative Agriculture and Food  
        **33 Thomas Rippel, using a blockchain to help Farmland Stewardship Organisations grow**  
        **0:37:40-0:38:18**   
        it you know they're so idealistic they would never do that. But let's think one generation down the line. They inherit these shares you know they have no personal connection to this anymore. And then then you're really facing danger. How do you get around them. Well there's a very interesting model also in **germany** by a syndicate that has found a legal way to get around this in that. So they're doing this for real estate for housing. And there's OK. How can I put this really simply.  
        ---

Investing in Regenerative Agriculture and Food  
        **33 Thomas Rippel, using a blockchain to help Farmland Stewardship Organisations grow**  
        **0:45:39-0:46:49**   
        out to sustainable farmers right so where would you be buying so I'm buying these tokens and we're not going to go into all the technical details I will definitely post a link to the project in the description but so I'm buying the tokens where would terra fina buy the land is that Switzerland and bound because you're there or is it much more freely like it could also be **germany** or France or even further further abroad yeah so we are really looking at first at **germany** and France because there's these very well established farmland stewardship organizations that have the three that I'm looking at terra de lien in France that has existed for 10 years and been implementing this model of buying land and through and raising money through issued shares that offer no return so you would be partnering with with the with already established farmland stewardship organizations and helping them  
        ---

Investing in Regenerative Agriculture and Food  
        **33 Thomas Rippel, using a blockchain to help Farmland Stewardship Organisations grow**  
        **0:46:07-0:47:12**   
        looking at first at **germany** and France because there's these very well established farmland stewardship organizations that have the three that I'm looking at terra de lien in France that has existed for 10 years and been implementing this model of buying land and through and raising money through issued shares that offer no return so you would be partnering with with the with already established farmland stewardship organizations and helping them basically to to grow right exactly so we would raise the capital through the issued tokens and then invest in those existing organizations that have a very very solid track record and those three organizations I'm looking at the B.O. Bodengen Ostershaft in **germany** Kulturland Genschlaff in Germany Kulturland Genschlaff that I have a very close relationship with and the board of directors has approved to  
        ---

...

Investing in Regenerative Agriculture and Food  
        **Transition Finance for Farmers with Benedikt - Gut&Bösel farm**  
        **0:07:19-0:07:57**   
        actually implementing one of these projects. Or am I wrong? Yeah. It took some motivation, I guess, to bring him over. But I mean, his way of doing agriculture is quite popular in Brazil, in Argentina, and also on a large scale. So it is being proven in a way there already. And he started a couple of projects in Europe, one in Spain, one in Switzerland, one in Portugal, and a few others. And then we are lucky to have him in **germany** as well.  
        ---

Farmerama  
        **44: Radical roots, Slow Money, CSAs and Allies.**  
        **0:19:00-0:19:57**   
        and farming too, why is it so important to have a CSA cross-border event at this time? I told myself this morning I wouldn't use the word Brexit today. CSA beyond borders is not just about beyond the border, you know, whether hard or soft or whatever it is on the island of Ireland, but across the world. And this, Marasa and I met in Thessaloniki at a community-supported agriculture international event organised by Roisin and it was there that we plotted the idea to have this event. And my mother last week went to a CSA in **germany** next to where my brother lives in **germany** and the welcome that she received there and the on-the-ground effort to provide food, to live in a community where people have real relationships based on trust, on resilience and that kind of thing.  
        ---

Farmerama  
        **44: Radical roots, Slow Money, CSAs and Allies.**  
        **0:19:27-0:20:15**   
        event organised by Roisin and it was there that we plotted the idea to have this event. And my mother last week went to a CSA in **germany** next to where my brother lives in **germany** and the welcome that she received there and the on-the-ground effort to provide food, to live in a community where people have real relationships based on trust, on resilience and that kind of thing. And we experienced it last night, staying with Jenny and her hospitality and meeting you this morning. And we know it from other CSAs that we've visited, be it in Ireland, be it in the UK, be it in Scotland, it doesn't matter what the names of the places are, the people are always the same.  
        ---

Farmerama  
        **61: Fine Fleeces, Pasture Cropping and Land in Commons**  
        **0:36:53-0:38:09**   
        and we are just now in the final stages. In the next couple of weeks we will make the final agreements with the two couples who will be taking over this farm and we now have several farms that we are in the process of taking over and then looking for young people to farm. I think the way we have worked it out now is very much scalable without running the danger that it is going to get structurally corrupted. I think we put in a lot of safeguards there. I really do think that this can be a model that can be a solution for thousands of farms in **germany** but also outside of **germany** and I am very happy to share our know-how. I think it can be pretty much copy-pasted and also the infrastructure that we created in terms of the IT, we have worked very, very hard to keep developing that. It is still in prototype phase but by the middle of next year we should have a 2.0 version of the software  
        ---

Farmerama  
        **61: Fine Fleeces, Pasture Cropping and Land in Commons**  
        **0:37:27-0:38:45**   
        there. I really do think that this can be a model that can be a solution for thousands of farms in **germany** but also outside of **germany** and I am very happy to share our know-how. I think it can be pretty much copy-pasted and also the infrastructure that we created in terms of the IT, we have worked very, very hard to keep developing that. It is still in prototype phase but by the middle of next year we should have a 2.0 version of the software ready where other users can use it. Individual farms could use the software just to run their own little crowd investing campaign. We are not going to charge the classical fee model of whatever, 5-7% or so on. We want this to be a commons-based tool. We are going to say, well, we think maybe we need about 3% or something like that to cover our costs but then it is up to you to decide if you can pay more or less than that.  
        ---